In [10]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, ConfusionMatrixDisplay, confusion_matrix
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomZoom, Resizing, Rescaling
from sklearn.cluster import KMeans

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import shutil

from sklearn.utils import class_weight
from eval_scores import scores

In [11]:
# Extract data
X = np.load('Xtrain_Classification_Part2.npy')
y = np.load('Ytrain_Classification_Part2.npy')

bacc_aug_data = 0
bacc_clusters_a_data = 0
bacc_clusters = 0
bacc_weights_a_aug_data = 0
bacc_weights_a_partially_aug_data_african = 0
bacc_weights_a_partially_aug_data = 0
bacc_weights = 0
bacc_cnn = 0

In [12]:
for i in range(10):

    # Split in train and validate 
    X_train, X_validate, train_labels, test_labels = train_test_split(X,y)


    # Reshape to 50x50 pixel images
    pixels = 50
    train_images = []
    test_images = []

    for image in X_train:
        train_images.append(image.reshape(pixels, pixels))

    for image in X_validate:
        test_images.append(image.reshape(pixels, pixels))

    train_images = np.array(train_images)
    test_images = np.array(test_images)


    # Normalizing
    train_images = train_images / 255.0
    test_images = test_images / 255.0

    X_train = X_train / 255.0
    X_validate = X_validate / 255.0


    # Sort data based on ethnicity
    caucasian_count = 0
    african_count = 0
    asian_count = 0
    indian_count = 0

    caucasian_train = []
    african_train = []
    asian_train = []
    indian_train = []

    for index, label in enumerate(train_labels):
        if label == 0:
            caucasian_count += 1
            caucasian_train.append(train_images[index])
        elif label == 1:
            african_count += 1
            african_train.append(train_images[index])
        elif label == 2:
            asian_count += 1
            asian_train.append(train_images[index])
        elif label == 3:
            indian_count += 1
            indian_train.append(train_images[index])
        

    # Augment caucasian data by adding mirrored images
    caucasian_train = np.array(caucasian_train)
    caucasian_augmented = []

    caucasian_flipped = caucasian_train[:,:,::-1]

    for index, image in enumerate(caucasian_flipped):
        caucasian_augmented.append(caucasian_train[index])
        caucasian_augmented.append(image)

    caucasian_augmented = np.array(caucasian_augmented)


    # Augment african data by adding mirrored images
    african_train = np.array(african_train)
    african_augmented = []

    african_flipped = african_train[:,:,::-1]

    for index, image in enumerate(african_flipped):
        african_augmented.append(african_train[index])
        african_augmented.append(image)

    african_augmented = np.array(african_augmented)


    # Augment asian data by adding mirrored images
    asian_train = np.array(asian_train)
    asian_augmented = []

    asian_flipped = asian_train[:,:,::-1]

    for index, image in enumerate(asian_flipped):
        asian_augmented.append(asian_train[index])
        asian_augmented.append(image)

    asian_augmented = np.array(asian_augmented)


    # Augment indian data by adding mirrored images
    indian_train = np.array(indian_train)
    indian_augmented = []

    indian_flipped = indian_train[:,:,::-1]

    for index, image in enumerate(indian_flipped):
        indian_augmented.append(indian_train[index])
        indian_augmented.append(image)

    indian_augmented = np.array(indian_augmented)


    # Create clusters
    k = max(african_count, max(asian_count, indian_count))
    clusters = KMeans(n_clusters = k, max_iter = 100, n_init = 5, random_state = 20)
    caucasian_train = np.array(caucasian_train)
    reshaped_data = caucasian_train.reshape(len(caucasian_train),-1)
    clusters_caucasian = clusters.fit_predict(reshaped_data)
    clusters_caucasian = clusters.cluster_centers_

    caucasian_cluster_array = []
    for image in clusters_caucasian:
        caucasian_cluster_array.append(image.reshape(pixels, pixels))
    caucasian_cluster_array = np.array(caucasian_cluster_array)


    # Create new training data
    train_images_aug_data = []
    train_labels_aug_data = []
    train_images_clusters_a_data = []
    train_labels_clusters_a_data = []
    train_images_clusters = []
    train_labels_clusters = []
    train_images_weights_a_aug_data = []
    train_labels_weights_a_aug_data = []
    train_images_weights_a_partially_aug_data_african = []
    train_labels_weights_a_partially_aug_data_african = []
    train_images_weights_a_partially_aug_data = []
    train_labels_weights_a_partially_aug_data = []

    train_images_weights = train_images
    train_labels_weights = train_labels

    count_caucasian = 0
    count_african = 0
    count_asian = 0
    count_indian = 0

    for index, label in enumerate(train_labels):
        if label == 0:
            # Aug data
            train_images_aug_data.append(caucasian_augmented[count_caucasian].reshape(pixels,pixels))
            train_labels_aug_data.append(label)
            train_images_aug_data.append(caucasian_augmented[count_caucasian + caucasian_count].reshape(pixels,pixels))
            train_labels_aug_data.append(label)

            if count_caucasian < len(caucasian_cluster_array):
                # Clusters a data
                train_images_clusters_a_data.append(caucasian_cluster_array[count_caucasian].reshape(pixels,pixels))
                train_labels_clusters_a_data.append(label)

                # Clusters
                train_images_clusters.append(caucasian_cluster_array[count_caucasian].reshape(pixels,pixels))
                train_labels_clusters.append(label)

            # Weights a aug data
            train_images_weights_a_aug_data.append(caucasian_augmented[count_caucasian].reshape(pixels,pixels))
            train_labels_weights_a_aug_data.append(label)
            train_images_weights_a_aug_data.append(caucasian_augmented[count_caucasian + caucasian_count].reshape(pixels,pixels))
            train_labels_weights_a_aug_data.append(label)

            # Weights a partially aug data african
            train_images_weights_a_partially_aug_data_african.append(train_images[index].reshape(pixels,pixels))
            train_labels_weights_a_partially_aug_data_african.append(label)

            # Weights a partially aug data
            train_images_weights_a_partially_aug_data.append(train_images[index].reshape(pixels,pixels))
            train_labels_weights_a_partially_aug_data.append(label)

            count_caucasian += 1

        elif label == 1:
            # Aug data
            train_images_aug_data.append(african_augmented[count_african].reshape(pixels,pixels))
            train_labels_aug_data.append(label)
            train_images_aug_data.append(african_augmented[count_african + african_count].reshape(pixels,pixels))
            train_labels_aug_data.append(label)

            # Clusters a data
            train_images_clusters_a_data.append(african_augmented[count_african].reshape(pixels,pixels))
            train_labels_clusters_a_data.append(label)
            train_images_clusters_a_data.append(african_augmented[count_african + african_count].reshape(pixels,pixels))
            train_labels_clusters_a_data.append(label)

            # Clusters
            train_images_clusters.append(train_images[index].reshape(pixels,pixels))
            train_labels_clusters.append(label)

            # Weights a aug data
            train_images_weights_a_aug_data.append(african_augmented[count_african].reshape(pixels,pixels))
            train_labels_weights_a_aug_data.append(label)
            train_images_weights_a_aug_data.append(african_augmented[count_african + african_count].reshape(pixels,pixels))
            train_labels_weights_a_aug_data.append(label)

            # Weights a partially aug data african
            train_images_weights_a_partially_aug_data_african.append(african_augmented[count_african].reshape(pixels,pixels))
            train_labels_weights_a_partially_aug_data_african.append(label)
            train_images_weights_a_partially_aug_data_african.append(african_augmented[count_african + african_count].reshape(pixels,pixels))
            train_labels_weights_a_partially_aug_data_african.append(label)

            # Weights a partially aug data
            train_images_weights_a_partially_aug_data.append(african_augmented[count_african].reshape(pixels,pixels))
            train_labels_weights_a_partially_aug_data.append(label)
            train_images_weights_a_partially_aug_data.append(african_augmented[count_african + african_count].reshape(pixels,pixels))
            train_labels_weights_a_partially_aug_data.append(label)

            count_african += 1

        elif label == 2:
            # Aug data
            train_images_aug_data.append(asian_augmented[count_asian].reshape(pixels,pixels))
            train_labels_aug_data.append(label)
            train_images_aug_data.append(asian_augmented[count_asian + asian_count].reshape(pixels,pixels))
            train_labels_aug_data.append(label)

            # Clusters a data
            train_images_clusters_a_data.append(train_images[index].reshape(pixels,pixels))
            train_labels_clusters_a_data.append(label)

            # Clusters
            train_images_clusters.append(train_images[index].reshape(pixels,pixels))
            train_labels_clusters.append(label)

            # Weights a aug data
            train_images_weights_a_aug_data.append(asian_augmented[count_asian].reshape(pixels,pixels))
            train_labels_weights_a_aug_data.append(label)
            train_images_weights_a_aug_data.append(asian_augmented[count_asian + asian_count].reshape(pixels,pixels))
            train_labels_weights_a_aug_data.append(label)

            # Weights a partially aug data african
            train_images_weights_a_partially_aug_data_african.append(train_images[index].reshape(pixels,pixels))
            train_labels_weights_a_partially_aug_data_african.append(label)

            # Weights a partially aug data
            train_images_weights_a_partially_aug_data.append(asian_augmented[count_asian].reshape(pixels,pixels))
            train_labels_weights_a_partially_aug_data.append(label)
            train_images_weights_a_partially_aug_data.append(asian_augmented[count_asian + asian_count].reshape(pixels,pixels))
            train_labels_weights_a_partially_aug_data.append(label)

            count_asian += 1

        elif label == 3:
            # Aug data
            train_images_aug_data.append(indian_augmented[count_indian].reshape(pixels,pixels))
            train_labels_aug_data.append(label)
            train_images_aug_data.append(indian_augmented[count_indian + indian_count].reshape(pixels,pixels))
            train_labels_aug_data.append(label)

            # Clusters a data
            train_images_clusters_a_data.append(train_images[index].reshape(pixels,pixels))
            train_labels_clusters_a_data.append(label)

            # Clusters
            train_images_clusters.append(train_images[index].reshape(pixels,pixels))
            train_labels_clusters.append(label)

            # Weights a aug data
            train_images_weights_a_aug_data.append(indian_augmented[count_indian].reshape(pixels,pixels))
            train_labels_weights_a_aug_data.append(label)
            train_images_weights_a_aug_data.append(indian_augmented[count_indian + indian_count].reshape(pixels,pixels))
            train_labels_weights_a_aug_data.append(label)

            # Weights a partially aug data african
            train_images_weights_a_partially_aug_data_african.append(train_images[index].reshape(pixels,pixels))
            train_labels_weights_a_partially_aug_data_african.append(label)

            # Weights a partially aug data
            train_images_weights_a_partially_aug_data.append(indian_augmented[count_indian].reshape(pixels,pixels))
            train_labels_weights_a_partially_aug_data.append(label)
            train_images_weights_a_partially_aug_data.append(indian_augmented[count_indian + indian_count].reshape(pixels,pixels))
            train_labels_weights_a_partially_aug_data.append(label)

            count_indian += 1

    # Cast to array
    train_images_aug_data = np.array(train_images_aug_data)
    train_labels_aug_data = np.array(train_labels_aug_data)
    train_images_clusters_a_data = np.array(train_images_clusters_a_data)
    train_labels_clusters_a_data = np.array(train_labels_clusters_a_data)
    train_images_clusters = np.array(train_images_clusters)
    train_labels_clusters = np.array(train_labels_clusters)
    train_images_weights_a_aug_data = np.array(train_images_weights_a_aug_data)
    train_labels_weights_a_aug_data = np.array(train_labels_weights_a_aug_data)
    train_images_weights_a_partially_aug_data_african = np.array(train_images_weights_a_partially_aug_data_african)
    train_labels_weights_a_partially_aug_data_african = np.array(train_labels_weights_a_partially_aug_data_african)
    train_images_weights_a_partially_aug_data = np.array(train_images_weights_a_partially_aug_data)
    train_labels_weights_a_partially_aug_data = np.array(train_labels_weights_a_partially_aug_data)



    # Build models and train models

    # CNN ---------------------------------------------------------------------------------------------------------
    model = keras.Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(50, 50, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(4, activation="softmax"))

    model.compile(  optimizer='adam',
                    loss = 'sparse_categorical_crossentropy', #'binary_crossentropy',
                    metrics = ['accuracy'])
    
    # Train model
    train_images = np.reshape(train_images, (len(train_images), 50, 50, 1))
    test_images = np.reshape(test_images, (len(test_images),50, 50, 1))

    history = model.fit(train_images, train_labels, validation_data = (test_images, test_labels), epochs = 10)

    test_images = test_images.reshape(-1, 50, 50, 1)

    predictions = model.predict(test_images)
    y_predicted = []
    for i in predictions:
        y_predicted.append(np.argmax(i))
    y_predicted = np.array(y_predicted)

    bacc_cnn += scores(test_labels, y_predicted, 'c')

    # CNN w aug data ---------------------------------------------------------------------------------------------------------
    model_w_aug_data = keras.Sequential()
    model_w_aug_data.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(50, 50, 1)))
    model_w_aug_data.add(MaxPooling2D((2, 2)))
    model_w_aug_data.add(Dropout(0.2))
    model_w_aug_data.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_w_aug_data.add(MaxPooling2D((2, 2)))
    model_w_aug_data.add(Dropout(0.2))
    model_w_aug_data.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_w_aug_data.add(Flatten())
    model_w_aug_data.add(Dense(64, activation='relu'))
    model_w_aug_data.add(Dropout(0.2))
    model_w_aug_data.add(Dense(4, activation="softmax"))

    model_w_aug_data.compile(  optimizer='adam',
                                loss = 'sparse_categorical_crossentropy', #'binary_crossentropy',
                                metrics = ['accuracy'])
    
    # Train model
    train_images_aug_data = np.reshape(train_images_aug_data, (len(train_images_aug_data), 50, 50, 1))
    train_images_aug_data = np.asarray(train_images_aug_data)
    train_labels_aug_data = np.asarray(train_labels_aug_data)

    test_images = np.reshape(test_images, (len(test_images),50, 50, 1))

    history = model_w_aug_data.fit(train_images_aug_data, train_labels_aug_data, validation_data = (test_images, test_labels), epochs = 10)

    test_images = test_images.reshape(-1, 50, 50, 1)

    predictions_w_aug_data = model_w_aug_data.predict(test_images)
    y_predicted_w_aug_data = []
    for i in predictions_w_aug_data:
        y_predicted_w_aug_data.append(np.argmax(i))
    y_predicted_w_aug_data = np.array(y_predicted_w_aug_data)

    bacc_aug_data += scores(test_labels, y_predicted_w_aug_data, 'c')


    # CNN w clusters a aug data ---------------------------------------------------------------------------------------------------------
    model_clusters_a_data = keras.Sequential()
    model_clusters_a_data.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(50, 50, 1)))
    model_clusters_a_data.add(MaxPooling2D((2, 2)))
    model_clusters_a_data.add(Dropout(0.2))
    model_clusters_a_data.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_clusters_a_data.add(MaxPooling2D((2, 2)))
    model_clusters_a_data.add(Dropout(0.2))
    model_clusters_a_data.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_clusters_a_data.add(Flatten())
    model_clusters_a_data.add(Dense(64, activation='relu'))
    model_clusters_a_data.add(Dropout(0.2))
    model_clusters_a_data.add(Dense(4, activation="softmax"))

    model_clusters_a_data.compile(  optimizer='adam',
                                loss = 'sparse_categorical_crossentropy', #'binary_crossentropy',
                                metrics = ['accuracy'])
    
    # Train model
    train_images_clusters_a_data = np.reshape(train_images_clusters_a_data, (len(train_images_clusters_a_data), 50, 50, 1))
    train_images_clusters_a_data = np.asarray(train_images_clusters_a_data)
    train_labels_clusters_a_data = np.asarray(train_labels_clusters_a_data)

    test_images = np.reshape(test_images, (len(test_images),50, 50, 1))

    history = model_clusters_a_data.fit(train_images_clusters_a_data, train_labels_clusters_a_data, validation_data = (test_images, test_labels), epochs = 10)

    test_images = test_images.reshape(-1, 50, 50, 1)

    predictions_clusters_a_data = model_clusters_a_data.predict(test_images)
    y_predicted_clusters_a_data = []
    for i in predictions_clusters_a_data:
        y_predicted_clusters_a_data.append(np.argmax(i))
    y_predicted_clusters_a_data = np.array(y_predicted_clusters_a_data)

    bacc_clusters_a_data += scores(test_labels, y_predicted_clusters_a_data, 'c')


    # CNN w clusters ---------------------------------------------------------------------------------------------------------
    model_clusters = keras.Sequential()
    model_clusters.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(50, 50, 1)))
    model_clusters.add(MaxPooling2D((2, 2)))
    model_clusters.add(Dropout(0.2))
    model_clusters.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_clusters.add(MaxPooling2D((2, 2)))
    model_clusters.add(Dropout(0.2))
    model_clusters.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_clusters.add(Flatten())
    model_clusters.add(Dense(64, activation='relu'))
    model_clusters.add(Dropout(0.2))
    model_clusters.add(Dense(4, activation="softmax"))

    model_clusters.compile(  optimizer='adam',
                                loss = 'sparse_categorical_crossentropy', #'binary_crossentropy',
                                metrics = ['accuracy'])
    
    # Train model
    train_images_clusters = np.reshape(train_images_clusters, (len(train_images_clusters), 50, 50, 1))
    train_images_clusters = np.asarray(train_images_clusters)
    train_labels_clusters = np.asarray(train_labels_clusters)

    test_images = np.reshape(test_images, (len(test_images),50, 50, 1))

    history = model_clusters.fit(train_images_clusters, train_labels_clusters, validation_data = (test_images, test_labels), epochs = 10)

    test_images = test_images.reshape(-1, 50, 50, 1)

    predictions_clusters = model_clusters.predict(test_images)
    y_predicted_clusters = []
    for i in predictions_clusters:
        y_predicted_clusters.append(np.argmax(i))
    y_predicted_clusters = np.array(y_predicted_clusters)

    bacc_clusters += scores(test_labels, y_predicted_clusters, 'c')


    # CNN w weights a aug data ---------------------------------------------------------------------------------------------------------
    model_weights_a_aug_data = keras.Sequential()
    model_weights_a_aug_data.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(50, 50, 1)))
    model_weights_a_aug_data.add(MaxPooling2D((2, 2)))
    model_weights_a_aug_data.add(Dropout(0.2))
    model_weights_a_aug_data.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_weights_a_aug_data.add(MaxPooling2D((2, 2)))
    model_weights_a_aug_data.add(Dropout(0.2))
    model_weights_a_aug_data.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_weights_a_aug_data.add(Flatten())
    model_weights_a_aug_data.add(Dense(64, activation='relu'))
    model_weights_a_aug_data.add(Dropout(0.2))
    model_weights_a_aug_data.add(Dense(4, activation="softmax"))

    model_weights_a_aug_data.compile(  optimizer='adam',
                                loss = 'sparse_categorical_crossentropy', #'binary_crossentropy',
                                metrics = ['accuracy'])
    
    # Train model
    train_images_weights_a_aug_data = np.reshape(train_images_weights_a_aug_data, (len(train_images_weights_a_aug_data), 50, 50, 1))
    train_images_weights_a_aug_data = np.asarray(train_images_weights_a_aug_data)
    train_labels_weights_a_aug_data = np.asarray(train_labels_weights_a_aug_data)

    test_images = np.reshape(test_images, (len(test_images),50, 50, 1))

    # Calculate weights
    class_weights = class_weight.compute_class_weight('balanced', np.unique(train_labels_weights_a_aug_data), train_labels_weights_a_aug_data)
    classes = [0, 1, 2, 3]
    dict_weights = dict(zip(classes, class_weights.T))

    history = model_weights_a_aug_data.fit(train_images_weights_a_aug_data, train_labels_weights_a_aug_data, validation_data = (test_images, test_labels), epochs = 10, class_weight=dict_weights)

    test_images = test_images.reshape(-1, 50, 50, 1)

    predictions_weights_a_aug_data = model_weights_a_aug_data.predict(test_images)
    y_predicted_weights_a_aug_data = []
    for i in predictions_weights_a_aug_data:
        y_predicted_weights_a_aug_data.append(np.argmax(i))
    y_predicted_weights_a_aug_data = np.array(y_predicted_weights_a_aug_data)

    bacc_weights_a_aug_data += scores(test_labels, y_predicted_weights_a_aug_data, 'c')




    # CNN w weights a partially aug data african ---------------------------------------------------------------------------------------------------------
    model_weights_a_partially_aug_data_african = keras.Sequential()
    model_weights_a_partially_aug_data_african.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(50, 50, 1)))
    model_weights_a_partially_aug_data_african.add(MaxPooling2D((2, 2)))
    model_weights_a_partially_aug_data_african.add(Dropout(0.2))
    model_weights_a_partially_aug_data_african.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_weights_a_partially_aug_data_african.add(MaxPooling2D((2, 2)))
    model_weights_a_partially_aug_data_african.add(Dropout(0.2))
    model_weights_a_partially_aug_data_african.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_weights_a_partially_aug_data_african.add(Flatten())
    model_weights_a_partially_aug_data_african.add(Dense(64, activation='relu'))
    model_weights_a_partially_aug_data_african.add(Dropout(0.2))
    model_weights_a_partially_aug_data_african.add(Dense(4, activation="softmax"))

    model_weights_a_partially_aug_data_african.compile(  optimizer='adam',
                                loss = 'sparse_categorical_crossentropy', #'binary_crossentropy',
                                metrics = ['accuracy'])
    
    # Train model
    train_images_weights_a_partially_aug_data_african = np.reshape(train_images_weights_a_partially_aug_data_african, (len(train_images_weights_a_partially_aug_data_african), 50, 50, 1))
    train_images_weights_a_partially_aug_data_african = np.asarray(train_images_weights_a_partially_aug_data_african)
    train_labels_weights_a_partially_aug_data_african = np.asarray(train_labels_weights_a_partially_aug_data_african)

    test_images = np.reshape(test_images, (len(test_images),50, 50, 1))

    # Calculate weights
    class_weights = class_weight.compute_class_weight('balanced', np.unique(train_labels_weights_a_partially_aug_data_african), train_labels_weights_a_partially_aug_data_african)
    classes = [0, 1, 2, 3]
    dict_weights = dict(zip(classes, class_weights.T))

    history = model_weights_a_partially_aug_data_african.fit(train_images_weights_a_partially_aug_data_african, train_labels_weights_a_partially_aug_data_african, validation_data = (test_images, test_labels), epochs = 10, class_weight=dict_weights)

    test_images = test_images.reshape(-1, 50, 50, 1)

    predictions_weights_a_partially_aug_data_african = model_weights_a_partially_aug_data_african.predict(test_images)
    y_predicted_weights_a_partially_aug_data_african = []
    for i in predictions_weights_a_partially_aug_data_african:
        y_predicted_weights_a_partially_aug_data_african.append(np.argmax(i))
    y_predicted_weights_a_partially_aug_data_african = np.array(y_predicted_weights_a_partially_aug_data_african)

    bacc_weights_a_partially_aug_data_african += scores(test_labels, y_predicted_weights_a_partially_aug_data_african, 'c')


    # CNN w weights a partially aug data ---------------------------------------------------------------------------------------------------------
    model_weights_a_partially_aug_data = keras.Sequential()
    model_weights_a_partially_aug_data.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(50, 50, 1)))
    model_weights_a_partially_aug_data.add(MaxPooling2D((2, 2)))
    model_weights_a_partially_aug_data.add(Dropout(0.2))
    model_weights_a_partially_aug_data.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_weights_a_partially_aug_data.add(MaxPooling2D((2, 2)))
    model_weights_a_partially_aug_data.add(Dropout(0.2))
    model_weights_a_partially_aug_data.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_weights_a_partially_aug_data.add(Flatten())
    model_weights_a_partially_aug_data.add(Dense(64, activation='relu'))
    model_weights_a_partially_aug_data.add(Dropout(0.2))
    model_weights_a_partially_aug_data.add(Dense(4, activation="softmax"))

    model_weights_a_partially_aug_data.compile(  optimizer='adam',
                                loss = 'sparse_categorical_crossentropy', #'binary_crossentropy',
                                metrics = ['accuracy'])
    
    # Train model
    train_images_weights_a_partially_aug_data = np.reshape(train_images_weights_a_partially_aug_data, (len(train_images_weights_a_partially_aug_data), 50, 50, 1))
    train_images_weights_a_partially_aug_data = np.asarray(train_images_weights_a_partially_aug_data)
    train_labels_weights_a_partially_aug_data = np.asarray(train_labels_weights_a_partially_aug_data)

    test_images = np.reshape(test_images, (len(test_images),50, 50, 1))

    # Calculate weights
    class_weights = class_weight.compute_class_weight('balanced', np.unique(train_labels_weights_a_partially_aug_data), train_labels_weights_a_partially_aug_data)
    classes = [0, 1, 2, 3]
    dict_weights = dict(zip(classes, class_weights.T))

    history = model_weights_a_partially_aug_data.fit(train_images_weights_a_partially_aug_data, train_labels_weights_a_partially_aug_data, validation_data = (test_images, test_labels), epochs = 10, class_weight=dict_weights)

    test_images = test_images.reshape(-1, 50, 50, 1)

    predictions_weights_a_partially_aug_data = model_weights_a_partially_aug_data.predict(test_images)
    y_predicted_weights_a_partially_aug_data = []
    for i in predictions_weights_a_partially_aug_data:
        y_predicted_weights_a_partially_aug_data.append(np.argmax(i))
    y_predicted_weights_a_partially_aug_data = np.array(y_predicted_weights_a_partially_aug_data)

    bacc_weights_a_partially_aug_data += scores(test_labels, y_predicted_weights_a_partially_aug_data, 'c')


    # CNN w weights ---------------------------------------------------------------------------------------------------------
    model_weights = keras.Sequential()
    model_weights.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(50, 50, 1)))
    model_weights.add(MaxPooling2D((2, 2)))
    model_weights.add(Dropout(0.2))
    model_weights.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_weights.add(MaxPooling2D((2, 2)))
    model_weights.add(Dropout(0.2))
    model_weights.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model_weights.add(Flatten())
    model_weights.add(Dense(64, activation='relu'))
    model_weights.add(Dropout(0.2))
    model_weights.add(Dense(4, activation="softmax"))

    model_weights.compile(  optimizer='adam',
                                loss = 'sparse_categorical_crossentropy', #'binary_crossentropy',
                                metrics = ['accuracy'])
    
    # Train model
    train_images_weights = np.reshape(train_images_weights, (len(train_images_weights), 50, 50, 1))
    train_images_weights = np.asarray(train_images_weights)
    train_labels_weights = np.asarray(train_labels_weights)

    test_images = np.reshape(test_images, (len(test_images),50, 50, 1))

    # Calculate weights
    class_weights = class_weight.compute_class_weight('balanced', np.unique(train_labels_weights), train_labels_weights)
    classes = [0, 1, 2, 3]
    dict_weights = dict(zip(classes, class_weights.T))

    history = model_weights.fit(train_images_weights, train_labels_weights, validation_data = (test_images, test_labels), epochs = 10, class_weight=dict_weights)

    test_images = test_images.reshape(-1, 50, 50, 1)

    predictions_weights = model_weights.predict(test_images)
    y_predicted_weights = []
    for i in predictions_weights:
        y_predicted_weights.append(np.argmax(i))
    y_predicted_weights = np.array(y_predicted_weights)

    bacc_weights += scores(test_labels, y_predicted_weights, 'c')



Epoch 1/10
173/173 [==============================] - 27s 158ms/step - loss: 0.9834 - accuracy: 0.6182 - val_loss: 0.8294 - val_accuracy: 0.6585
Epoch 2/10
173/173 [==============================] - 27s 158ms/step - loss: 0.7631 - accuracy: 0.6971 - val_loss: 0.7056 - val_accuracy: 0.7101
Epoch 3/10
173/173 [==============================] - 27s 159ms/step - loss: 0.6475 - accuracy: 0.7601 - val_loss: 0.6311 - val_accuracy: 0.7693
Epoch 4/10
173/173 [==============================] - 26s 152ms/step - loss: 0.5753 - accuracy: 0.7873 - val_loss: 0.5797 - val_accuracy: 0.7828
Epoch 5/10
173/173 [==============================] - 24s 140ms/step - loss: 0.5347 - accuracy: 0.8038 - val_loss: 0.5331 - val_accuracy: 0.7986
Epoch 6/10
173/173 [==============================] - 22s 130ms/step - loss: 0.5039 - accuracy: 0.8121 - val_loss: 0.5246 - val_accuracy: 0.8035
Epoch 7/10
173/173 [==============================] - 23s 132ms/step - loss: 0.4587 - accuracy: 0.8307 - val_loss: 0.5018 - val_ac

C:\Users\HP\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0. 1. 2. 3.], y=[2. 2. 2. ... 0. 2. 2.] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


Epoch 1/10
346/346 [==============================] - 50s 143ms/step - loss: 1.1381 - accuracy: 0.4986 - val_loss: 0.9116 - val_accuracy: 0.6249
Epoch 2/10
346/346 [==============================] - 49s 142ms/step - loss: 0.8153 - accuracy: 0.6685 - val_loss: 0.7396 - val_accuracy: 0.7155
Epoch 3/10
346/346 [==============================] - 50s 144ms/step - loss: 0.7059 - accuracy: 0.7182 - val_loss: 0.6167 - val_accuracy: 0.7655
Epoch 4/10
346/346 [==============================] - 50s 145ms/step - loss: 0.6238 - accuracy: 0.7524 - val_loss: 0.5917 - val_accuracy: 0.7780
Epoch 5/10
346/346 [==============================] - 46s 134ms/step - loss: 0.5526 - accuracy: 0.7783 - val_loss: 0.6214 - val_accuracy: 0.7698
Epoch 6/10
346/346 [==============================] - 44s 126ms/step - loss: 0.5096 - accuracy: 0.7841 - val_loss: 0.5593 - val_accuracy: 0.7910
Epoch 7/10
346/346 [==============================] - 43s 124ms/step - loss: 0.4516 - accuracy: 0.8068 - val_loss: 0.5459 - val_ac

C:\Users\HP\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0. 1. 2. 3.], y=[2. 2. 3. ... 0. 0. 2.] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


Epoch 1/10
181/181 [==============================] - 24s 130ms/step - loss: 1.2958 - accuracy: 0.3776 - val_loss: 1.1749 - val_accuracy: 0.4902
Epoch 2/10
181/181 [==============================] - 23s 128ms/step - loss: 0.9757 - accuracy: 0.5892 - val_loss: 0.9033 - val_accuracy: 0.6276
Epoch 3/10
181/181 [==============================] - 24s 134ms/step - loss: 0.8298 - accuracy: 0.6495 - val_loss: 0.9095 - val_accuracy: 0.6211
Epoch 4/10
181/181 [==============================] - 23s 127ms/step - loss: 0.7535 - accuracy: 0.6836 - val_loss: 0.6483 - val_accuracy: 0.7649
Epoch 5/10
181/181 [==============================] - 24s 135ms/step - loss: 0.6700 - accuracy: 0.7251 - val_loss: 0.5973 - val_accuracy: 0.7866
Epoch 6/10
181/181 [==============================] - 28s 153ms/step - loss: 0.6208 - accuracy: 0.7442 - val_loss: 0.6909 - val_accuracy: 0.7318
Epoch 7/10
181/181 [==============================] - 26s 144ms/step - loss: 0.5657 - accuracy: 0.7632 - val_loss: 0.7098 - val_ac

...

Epoch 1/10
181/181 [==============================] - 17s 93ms/step - loss: 1.3259 - accuracy: 0.3372 - val_loss: 1.3915 - val_accuracy: 0.3051
Epoch 2/10
181/181 [==============================] - 17s 94ms/step - loss: 1.0512 - accuracy: 0.5338 - val_loss: 1.0799 - val_accuracy: 0.5418
Epoch 3/10
181/181 [==============================] - 17s 93ms/step - loss: 0.8763 - accuracy: 0.6266 - val_loss: 0.9910 - val_accuracy: 0.5885
Epoch 4/10
181/181 [==============================] - 17s 93ms/step - loss: 0.7709 - accuracy: 0.6706 - val_loss: 0.7329 - val_accuracy: 0.7215
Epoch 5/10
181/181 [==============================] - 17s 92ms/step - loss: 0.6909 - accuracy: 0.7097 - val_loss: 0.6957 - val_accuracy: 0.7389
Epoch 6/10
181/181 [==============================] - 17s 92ms/step - loss: 0.6263 - accuracy: 0.7431 - val_loss: 0.6354 - val_accuracy: 0.7720
Epoch 7/10
181/181 [==============================] - 17s 92ms/step - loss: 0.5766 - accuracy: 0.7604 - val_loss: 0.6271 - val_accuracy:

C:\Users\HP\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0. 1. 2. 3.], y=[3. 3. 0. ... 0. 0. 0.] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


Epoch 1/10
242/242 [==============================] - 22s 91ms/step - loss: 1.2465 - accuracy: 0.4173 - val_loss: 1.0694 - val_accuracy: 0.5684
Epoch 2/10
242/242 [==============================] - 22s 92ms/step - loss: 0.8925 - accuracy: 0.6392 - val_loss: 0.7573 - val_accuracy: 0.7144
Epoch 3/10
242/242 [==============================] - 22s 91ms/step - loss: 0.7410 - accuracy: 0.7094 - val_loss: 0.8126 - val_accuracy: 0.6721
Epoch 4/10
242/242 [==============================] - 22s 92ms/step - loss: 0.6529 - accuracy: 0.7512 - val_loss: 0.8024 - val_accuracy: 0.6878
Epoch 5/10
242/242 [==============================] - 22s 92ms/step - loss: 0.6066 - accuracy: 0.7651 - val_loss: 0.6586 - val_accuracy: 0.7557
Epoch 6/10
242/242 [==============================] - 22s 91ms/step - loss: 0.5263 - accuracy: 0.7980 - val_loss: 0.6816 - val_accuracy: 0.7416
Epoch 7/10
242/242 [==============================] - 23s 93ms/step - loss: 0.4717 - accuracy: 0.8151 - val_loss: 0.5936 - val_accuracy:

C:\Users\HP\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0. 1. 2. 3.], y=[3. 0. 0. ... 0. 0. 0.] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


Epoch 1/10
173/173 [==============================] - 16s 93ms/step - loss: 1.2878 - accuracy: 0.3664 - val_loss: 1.1518 - val_accuracy: 0.5130
Epoch 2/10
173/173 [==============================] - 16s 93ms/step - loss: 0.9250 - accuracy: 0.6008 - val_loss: 0.8438 - val_accuracy: 0.6775
Epoch 3/10
173/173 [==============================] - 16s 93ms/step - loss: 0.7965 - accuracy: 0.6684 - val_loss: 0.8565 - val_accuracy: 0.6813
Epoch 4/10
173/173 [==============================] - 16s 93ms/step - loss: 0.7033 - accuracy: 0.7131 - val_loss: 0.6930 - val_accuracy: 0.7372
Epoch 5/10
173/173 [==============================] - 17s 99ms/step - loss: 0.6511 - accuracy: 0.7353 - val_loss: 0.7430 - val_accuracy: 0.7106
Epoch 6/10
173/173 [==============================] - 16s 95ms/step - loss: 0.5807 - accuracy: 0.7717 - val_loss: 0.8509 - val_accuracy: 0.6634
Epoch 7/10
173/173 [==============================] - 16s 93ms/step - loss: 0.5374 - accuracy: 0.7717 - val_loss: 0.6923 - val_accuracy:

In [13]:
print("1: ", bacc_aug_data)
print("2: ", bacc_clusters_a_data)
print("3: ", bacc_clusters)
print("4: ", bacc_weights_a_aug_data)
print("5: ", bacc_weights_a_partially_aug_data_african)
print("6: ", bacc_weights_a_partially_aug_data)
print("7: ", bacc_weights)
print("8: ", bacc_cnn)

1:  7.531370863838108
2:  7.437541587544072
3:  7.311854538822871
4:  7.7999906472888245
5:  7.7303269775286605
6:  7.731663822406558
7:  7.611537014193435
8:  7.21758043472077
